## Create FHIR R4 CapStatement Resource


### Outline:

- Source excel with requirements
- pandas to convert in python Ordered Dict
- build json
- generate narrative using Jinja2 templates

### Prerequisites:

- Python 3.6 or greater

## STEP1: Choose Source Spreadsheet to use

*note:  Jupyteralab and widgets issues: see https://stackoverflow.com/questions/4
9542417/how-to-get-ipywidgets-working-in-jupyter-lab for solution 

In [ ]:
#******************** NEED TO UPDATE WHEN Adding new IGS ************************************************

in_path_list = [
        "---pick one below---",
        "temp_source_spreadsheets/test-spreadsheet.xlsx",
        "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/source/source_spreadsheets/uscore-client.xlsx",
        "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/source/source_spreadsheets/uscore-server.xlsx",
        '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/alert-initiator.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/notification-forwarder.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/alert-receiver.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/query-responder.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/query-requester.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Consumer_Client.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Reporter_Client.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Consumer_Server.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Producer_Client.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Producer_Server.xlsx',
         '//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Receiver_Server.xlsx',
         "//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_GIC_Reporter_Client.xlsx",
          "//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_GIC_Receiver_Server.xlsx",
         'C:/Users/Administrator/Downloads/plan-net-server.xlsx'
        ]

#======================US Core ===================================
# ----------spreadsheet source---------------
from IPython.display import display as Display, HTML, Markdown, Javascript
from ipywidgets import Dropdown
menu = Dropdown(
       options=in_path_list,
       description='Choose Spreadsheet Source file',
       style = {'description_width': 'initial',},
       layout={'width': 'initial'},
        )


menu

## STEP2:  *CLICK HERE* and then 'Select Run Selected Cell and All Below'  from menu bar to continue"

In [ ]:
xls = menu.value
xls

### Import FHIRClient and other libraries

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import bundle as B
from fhirclient.r4models import narrative as N
import fhirclient.models.identifier as I
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown, Javascript
import ipywidgets as widgets
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime, date
from jinja2 import Environment, FileSystemLoader, select_autoescape
from stringcase import snakecase, titlecase
#from itertools import zip_longest
from openpyxl import load_workbook
from commonmark import commonmark
from lxml import etree

####  Assign Global Variables

Here is where we assign all the global variables for this example such as the canonical base and project information

In [ ]:
fhir_base_url = 'http://hl7.org/fhir/'
f_jurisdiction =  CC.CodeableConcept({
      "coding" : [
        {
          "system" : "urn:iso:std:iso:3166",
          "code" : "US"
        }
      ]
    })

conf_url = 'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation'
combo_url = 'http://hl7.org/fhir/StructureDefinition/capabilitystatement-search-parameter-combination'

sp_specials = {'us-core-includeprovenance':'http://hl7.org/fhir/us/core/SearchParameter/us-core-includeprovenance'}  # dict to for SP to get right canonicals, may use spreadsheet or package file in future.

none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False']

sep_list = (',', ';', ' ', ', ', '; ')

f_now = D.FHIRDate(str(date.today()))
f_now.as_json()

#### Conformance Extension

In [ ]:
def get_conf(conf='MAY',as_dict=False):
    if as_dict:
        return [X.Extension(dict(
            url = conf_url,
            valueCode = conf
            )).as_json()]
    else:
        return [X.Extension(dict(
            url = conf_url,
            valueCode = conf
            ))]
        

### Addin Extensions

In [ ]:
def get_addin_ext(py_ext, json_ext):            
    print(py_ext)
    if json_ext:   # ie not ''
        addin_ext = X.Extension(loads(json_ext))            
        # addin_ext.extension =  get_conf('SHALL') violates invariant   - DONT USE           
        print(addin_ext)                 
        py_ext.append(addin_ext) # add in other extensions
    print(py_ext)
    return py_ext

### validate

In [ ]:
# *********************** validate Resource ********************************

def validate(r):

    #fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server = 'http://hapi.fhir.org/baseR4'
    fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1'
    
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/Questionnaire/$validate', params = params, headers = headers, data = dumps(r.as_json()))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Get Cap Statement input data

function to convert to dataframe series to namedtuple for easy peasy dot notation use.

#### first the config data

In [ ]:
df = read_excel(xls,'config',na_filter = False,index_col=0)  # use the index_col = 0 for setting the first rwo as the index

df

#### assign globals e.g. publisher parameter etc...

In [ ]:
#df.[Column].[row] to get a value df.loc[[row],[Column]] or df.at[[column],[col]] works too
df.Value.source #, df.loc['source','Value'], df.at['source' ,'Value']

In [ ]:

ig_source_path = df.Value.source
ig_package_tar_path =  df.Value.packagepath
# --------- ig specific variable -------------------
pre = df.Value.pre  # for Titles - not sure this is actually used
canon = df.Value.canon # don't forget the slash  - fix using os.join or path
#
publisher = df.Value.publisher
#
publisher_endpoint = dict(
                    system = df.Value.publishersystem,
                    value = df.Value.publishervalue,
                  )

pprint(publisher_endpoint)


#### Get IG Names

until able to support primitive extensions in pyfhir

In [ ]:
def get_igs():
    ig_dict = {}
    df_igs = read_excel(xls,'igs',na_filter = False)
    for ig in df_igs.itertuples(index=True):
        ig_dict[ig.uri] = (ig.name, ig.url)
        
    return ig_dict # TODO add conformance to this and display extension

ig_dict = get_igs()


[*ig_dict]

#### then the meta sheet

In [ ]:
df = read_excel(xls,'meta',na_filter = False)

df

#### Create NamedTuple from df to use dot notation

In [ ]:
d = dict(zip(df.Element, df.Value))
meta = namedtuple("Meta", d.keys())(*d.values())      
         
meta.id


### Create CS instance

In [ ]:

def get_sys_op():
    op_list = []
    df_op = read_excel(xls,'ops',na_filter = False)
    for i in df_op.itertuples(index=True):
        if i.type == 'system':
            op = CS.CapabilityStatementRestResourceOperation()
            op.name = i.name 
            op.definition = i.definition
            op.extension = get_conf(i.conf)           
            op_list.append(op.as_json())
    return op_list


def get_rest_ints():
    ri_list = []
    df_ri = read_excel(xls,'rest_interactions',na_filter = False)
    for i in df_ri.itertuples(index=True):
        ri = CS.CapabilityStatementRestInteraction()
        ri.code = i.code 
        ri.documentation = i.doc if i.doc not in none_list else None
        ri.extension = get_conf(i.conf)
        print(ri.as_json())
        ri_list.append(ri.as_json())
    return ri_list

'''# TODO add conformance to this and display extension when support fhir primitives in pyfhir
def get_igs():
    ig_list = []
    df_igs = read_excel(xls,'igs',na_filter = False)
    for ig in df_igs.itertuples(index=True):
        ig_list.append(ig.uri)
    return ig_list 
'''
def kebab_to_pascal(word):
    return ''.join(x.capitalize() for x in word.split('-'))

cs = CS.CapabilityStatement()
cs.id = meta.id
cs.url = f'{canon}CapabilityStatement/{meta.id}'
cs.version = meta.version
cs.name = f'{kebab_to_pascal(meta.id)}{cs.resource_type}'
cs.title = f'{titlecase(meta.id).replace("Us ", "US ")} {cs.resource_type}'
cs.status = 'active'

cs.experimental = False
cs.date = f_now  # as FHIRDate
cs.publisher = publisher
cs.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
cs.description = meta.description
cs.jurisdiction = [f_jurisdiction]
cs.kind = 'requirements'
cs.fhirVersion = meta.fhirVersion
cs.acceptUnknown = 'both'
cs.format = [
    "xml",
    "json"
  ]
cs.patchFormat = [
    "application/json-patch+json",
  ]
cs.implementationGuide = meta.ig.split(",") + [*ig_dict]
rest = CS.CapabilityStatementRest(dict(
    mode = meta.mode,
    documentation = meta.documentation,
    security = dict(
        description = meta.security
        ) if meta.security else None,
    interaction = get_rest_ints(),
    operation = get_sys_op()
    ))
cs.rest = [rest]


cs.as_json()

#### Then the list of IG profiles

In [ ]:
df = read_excel(xls,'profiles',na_filter = False)

df

#### add Resources

- read sheets for resource attributes, interaction attributes,  search attributes, profiles, and combo search parameters

In [ ]:
df_resources = read_excel(xls,'resources',na_filter = False)
df_resources = df_resources[df_resources.type.str[0] != '!']
df_resources

In [ ]:
df_profiles = read_excel(xls,'profiles',na_filter = False)  #df1 = df[df.Hostname.str[0] != "abc"]
df_profiles = df_profiles[df_profiles.Profile.str[0] != '!']
df_profiles

In [ ]:
df_i = read_excel(xls,'interactions',na_filter = False)
df_sp = read_excel(xls,'sps',na_filter = False)
df_combos = read_excel(xls,'sp_combos',na_filter = False)
df_op = read_excel(xls,'ops',na_filter = False)


def get_i(type):
    int_list = []
    for i in df_i.itertuples(index=True):
        #print(i.code, getattr(i,f'conf_{type}'))
        if getattr(i,f'conf_{type}') not in none_list:
            int  = CS.CapabilityStatementRestResourceInteraction()
            int.code = i.code
            try:
                int.documentation = getattr(i,f'doc_{type}') if getattr(i,f'doc_{type}') not in none_list else None
            except AttributeError:
                pass
            int.extension = get_conf(getattr(i,f'conf_{type}'))    
            int_list.append(int.as_json())
        
    return int_list


def get_sp(r_type):
    sp_list = []
    for i in df_sp.itertuples(index=True):
        if i.base == r_type:
            sp  = CS.CapabilityStatementRestResourceSearchParam()
            sp.name = i.code
            
            # TODO need to fix this to reference the package file to reconcile definition to names
            if i.code in sp_specials: #special case temp fix for us-core
                sp.definition = sp_specials[i.code]
            elif i.update == 'Y' or i.exists =='N':
                sp.definition = (f'{canon}SearchParameter/{pre.lower()}-{i.base.lower()}-{i.code.split("_")[-1]}')                  
            else:  # use base definition
                sp.definition = f'{fhir_base_url}SearchParameter/{i.base}-{i.code.split("_")[-1]}'  # removes the '_' for things like _id
                                 
            # print(sp.definition)
                                 
            sp.type = i.type
            sp.extension = get_conf(i.base_conf)
            #print(sp.as_json())                
            sp_list.append(sp.as_json())
                             
    return sp_list


def get_combo_ext(r_type,combos):
    x_list = []
    for combo in combos:
        # convert to extension
        combo_ext = X.Extension()
        combo_ext.url = combo_url
        combo_conf_ext = get_conf(combo[1])
        combo_ext.extension=combo_conf_ext
        for param in combo[0].split(','):
            req_combo = X.Extension(
                dict (
                    url = 'required',
                    valueString = param   #http://hl7.org/fhir/us/core/SearchParameter/us-core-patient-family
                    )
                )
            combo_ext.extension.append(req_combo)
        x_list.append(combo_ext)
        # print(x_list)
    return x_list
                             
def get_op(r_type):
    op_list = []
    for i in df_op.itertuples(index=True):
         if i.type == r_type:
            op = CS.CapabilityStatementRestResourceOperation()
            op.name = i.name 
            op.definition = i.definition
            op.documentation = i.documentation if i.documentation not in none_list else None
            op.extension = get_conf(i.conf)
            try:                     
                op.extension =  get_addin_ext(op.extension, i.ext)
            except AttributeError:
                print("---- no addin extensions found-----")
            op_list.append(op.as_json())
                           
    return op_list 

rest.resource =  []
for r in df_resources.itertuples(index=True):
    # print(r.type, r.conformance, r.readHistory)
    supported_profile = [p.Profile for p in df_profiles.itertuples(index=True) if p.Type == r.type]
    #pprint(supported_profile)                         
    res = CS.CapabilityStatementRestResource(
    dict(
        type = r.type,
        documentation = r.documentation if r.documentation not in none_list else None,
        versioning = r.versioning if r.versioning not in none_list else None,
        readHistory = r.readHistory if r.readHistory not in none_list else None,
        updateCreate = r.updateCreate if r.updateCreate not in none_list else None,
        conditionalCreate = r.conditionalCreate if r.conditionalCreate not in none_list else None,
        conditionalRead = r.conditionalRead if r.conditionalRead not in none_list else None,
        conditionalUpdate = r.conditionalUpdate if r.conditionalUpdate not in none_list else None,
        conditionalDelete = r.conditionalDelete if r.conditionalDelete not in none_list else None,
        referencePolicy = [x for x in r.referencePolicy.split(",") if x],
        searchInclude =  [x for x in r.shall_include.split(",") + r.should_include.split(",") if x],
        searchRevInclude =  [x for x in r.shall_revinclude.split(",") + r.should_revinclude.split(",") if x],
        interaction = get_i(r.type),
        searchParam = get_sp(r.type),
        operation = get_op(r.type),
        profile = r.profile if r.profile not in none_list else None,
        supportedProfile = supported_profile,
        )
    )
    res.extension = get_conf(r.conformance)
    combos = {(i.combo,i.combo_conf) for i in df_combos.itertuples(index=True) if i.base == r.type}
    res.extension = res.extension + get_combo_ext(r.type,combos) # convert list to  lst of combo extension


    '''
    #TODO add in conformance expectations for primitives 
    #need to convert to dict since model can't handle primitive extensions

    resttype_dict = res.as_json()

    for i in ['Include','RevInclude']:
        element = f'_search{i}'

        resttype_dict[element] = []
        print(element)
        for expectation in ['should', 'shall']: # list all should includes first
            sp_attr = f'{expectation}_{i.lower()}'
            print(sp_attr) 
            includes = getattr(r,sp_attr).split(',')
            print(includes)

            for include in includes:
                if include not in none_list:             
                    print(include)
                    conf = get_conf(expectation.upper(),as_dict=True)
                    print(conf)
                    conf = conf
                    print(conf)        
                    resttype_dict[element].append(conf)

        if not resttype_dict[element]:
                del(resttype_dict[element])

    print(dumps(resttype_dict, indent = 4))
    res = CS.CapabilityStatementRestResource(resttype_dict, strict = False)
    print('++++++++++++++++RES.__dict__+++++++++++++++++++')
    print(dumps(res._searchRevInclude, indent = 4))
    '''                               

    rest.resource.append(res)

rest.resource =  sorted(rest.resource,key = lambda x: x.type)  # sort resources                         
cs.rest = [rest]
    
print(dumps(cs.as_json(),indent=3))    
        
        

### Convert model to dict and add extensions to primitives **Deactivated ( marked a raw block ) since will need to use dict in subsuquent steps.

### Validate

In [ ]:
 #validate and write to file

print('...validating')
r = validate(cs)
display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))



### Create Narrative

- Using Jinja2 Template create xhtml for narrative

#### First: Get spec_internal from package.tgz a json file which includes canonical to local relative page links

Note for this to work you have to have a working build that already contains all the needed artifacts.

In [ ]:
import tarfile
package_path = Path.cwd() / 'tarfiles'/'package.tgz'  #get_si(path)

def get_si(package_path):
    with tarfile.open(package_path, mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('package/other/spec.internals')
        r = f.read()
        si = loads(r)
        return si

    
def get_si3(path):
    tf = get(f'{path}/package.tgz')
    return tf

"e.g. https://build.fhir.org/ig/HL7/davinci-deqm/package.tgz" 
try:   
    tf= get_si3(ig_package_tar_path) # get from remote server
except:
   in_path = Path() / ig_package_tar_path /'package.tgz'
   tf = in_path.read_bytes()
   package_path.write_bytes(tf)  # get from package (json) file in local .fhir directory
else:
    package_path.write_bytes(tf.content)    #save in temp file
    
si = get_si(package_path) #unpack from file

path_map = si['paths']
path_map

#### Then Use Jinja2 template to create narrative

In [ ]:
in_path = ''
in_file = 'R4capabilitystatement-server.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)



env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

env.filters['markdown'] = markdown


template = env.get_template(in_file)

sp_map = {sp.code:sp.type for sp in df_sp.itertuples(index=True)}
pname_map = {p.Profile:p.Name for p in df_profiles.itertuples(index=True)}
pprint(pname_map)
rendered = template.render(cs=cs, path_map=path_map, pname_map=pname_map, sp_map=sp_map, ig_dict=ig_dict )
print(type(rendered))
display(HTML(rendered))


#======== write to temp file to debug =======
path = Path.cwd() / 'debug' / 'narrative.xhtml'
path.write_text(rendered, encoding="utf-8")
#===================================================

parser = etree.XMLParser(remove_blank_text=True)
root = etree.fromstring(rendered, parser=parser)

div = (etree.tostring(root[1][0], encoding='unicode', method='html'))
narr = N.Narrative()
narr.status = 'generated'
narr.div = div
cs.text = narr


#print(dumps(cs.as_json(),indent=3))

### validate again

In [ ]:
print('...validating')
r = validate(cs)
d = display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))
           
#======== write to temp file to debug =======
from html.parser import HTMLParser

class HTMLFilter(HTMLParser):
    text = ""
    def handle_data(self, data):
        self.text += data

f = HTMLFilter()
f.feed(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}')
path = Path.cwd() / 'debug' / 'validation.txt'
path.write_text(f.text)
#===================================================


### Write to folder

In [110]:
# save to file
#save in ig_source folder
path = Path.cwd() / ig_source_path / 'resources' / f'capabilitystatement-{cs.id.lower()}.json'

#path = Path.cwd() /  'resources' / f'capabilitystatement-{cs.id.lower()}.json' # write locally 


print(f'...........saving to file {path}............')
path.write_text(dumps(cs.as_json(), indent=4))

...........saving to file /Users/ehaas/Documents/Python/MyNotebooks/CapStatement/resources/capabilitystatement-test.json............


276611

#### 